In [ ]:
!pip install adversarial-robustness-toolbox
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from datetime import datetime 
from tqdm.notebook import tqdm 
import statistics
from math import log10
import struct
from random import randrange
import multiprocessing
import concurrent.futures
import time

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.preprocessing import QuantileTransformer, PowerTransformer

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib 


train_size = 10000

filename = '../dataset/purchase_member.csv'
df_train = pd.read_csv (filename, header=None,   sep='\t', encoding='utf-8') 
filename = '../dataset/purchase_non_member.csv'
df_test = pd.read_csv (filename,  header=None,  sep='\t', encoding='utf-8') 

print(df_train.shape, df_test.shape, f'{len(df_train)/(len(df_train)+len(df_test)):0.2f}')


#separating label from data and converting dataframe into Torch Tensor
col_0 = df_train.columns[0] # 1st column is label; 
col_rest = df_train.columns[1:] # rests are data
X_train = torch.tensor(df_train[col_rest].values, dtype=torch.float32) 
y_train = torch.tensor(df_train[col_0].values) # y is row vector here

print(f'-'*30, 'train', f'-'*30)
display(X_train.size(), y_train.size())
display(X_train, y_train)



# PurchaseClassifier model

class PurchaseClassifier(nn.Module):

    def __init__(self, num_features = 600, num_classes=100):
        super(PurchaseClassifier, self).__init__() 
        self.fc1 = nn.Linear(num_features,1024)
        self.fc2 = nn.Linear(1024,512)
        self.fc3 = nn.Linear(512,256)
        self.fc4 = nn.Linear(256,128)
        self.fc5 = nn.Linear(128,num_classes)
        self.relu = nn.Tanh()

    def forward(self, x):
       #classifier
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x) 
        x = self.relu(x)
        x = self.fc3(x) 
        x = self.relu(x)
        x = self.fc4(x)
        x = self.relu(x)

        x = self.fc5(x)

        # #sigmoid returns a value between 0 and 1, used for binary classification
        # prob = torch.sigmoid(x)   
        
        logits = x
        return logits



target_model = PurchaseClassifier()

model_path = '../dataset/Customer100.pth'
target_model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
target_model.eval()




def get_accuracy(model, data_loader):
    '''
    Function for computing the accuracy of the predictions over the entire data_loader
    '''
    
    correct_pred = 0 
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:

            y_hat = model(X)
            _, predicted_labels = torch.max(y_hat, 1)

            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()

    return correct_pred.float() / n




# optimizer = torch.optim.SGD(lr=args.lr)

# optimizer = torch.optim.Adam(target_model.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

y_hat = target_model(X_train)
loss = criterion(y_hat, y_train)
loss.backward()  # Now p.grad for this x is filled

# Need to clone it to save it
per_sample_gradients = [p.grad.detach().clone() for p in target_model.parameters()]

# all_per_sample_gradients.append(per_sample_gradients)
# model.zero_grad()  # p.grad is cumulative so we'd better reset it


for p in target_model.parameters():
  print(p) 
  print('\n\n\n') 


#separating label from data and converting dataframe into Torch Tensor
col_0 = df_test.columns[0] # 1st column is label; 
col_rest = df_test.columns[1:] # rests are data
X_test = torch.tensor(df_test[col_rest].values, dtype=torch.float32) 
y_test = torch.tensor(df_test[col_0].values) # y is row vector here

print(f'-'*30, 'test', f'-'*30)
# display(X_test.size(), y_test.size())
# display(X_test, y_test)


y_hat = target_model(X_test)
_, predicted_labels = torch.max(y_hat, 1)

n = y_test.size(0)
correct_pred = (predicted_labels == y_test).sum()
test_acc = correct_pred.float() / n
print(f'\nTarget model Test accuracy: {100 * test_acc:.2f}')




'''
I_bb MIA
'''

df_test = df_test.head(int(len(df_test)/5000) * (5000))
#split df_test into int(len(df_test)/5000) = 37 data frames, each having 5000 records;
df_test_list = np.split(df_test, len(df_test)/5000)
df_member = df_train.head(int(len(df_train)/2))
df_non_member = df_test_list[0] # taking 1st 5000 test data
df_mem_nonmem = pd.concat([df_member, df_non_member], ignore_index=True)

# print(df_member.shape, df_non_member.shape, f'{len(df_member)/(len(df_member)+len(df_non_member)):0.2f}')
# separating features: 1st column is label, rests are features
# col_0 = df_mem_nonmem.columns[0]
# y_mem_nonmem = torch.tensor(df_mem_nonmem[col_0].values)

col_rest = df_mem_nonmem.columns[1:]
X_mem_nonmem = torch.tensor(df_mem_nonmem[col_rest].values, dtype=torch.float32)

mem_y = np.array([int(1) for i in range(len(df_member))])
nonmem_y = np.array([int(0) for i in range(len(df_non_member))])
y_train_attacker_np = np.concatenate((mem_y, nonmem_y))


logit = target_model(X_mem_nonmem) #logit is tensor here
df_logit = pd.DataFrame(logit.detach().numpy())
scaler = StandardScaler() #94 70
# scaler = RobustScaler() #94 70
X_train_attacker_df  = pd.DataFrame(scaler.fit_transform(df_logit))


#=============== Attacker Training ==================
n_feature = len(X_train_attacker_df.columns) 
attacker_mlp_logit = MLPClassifier(hidden_layer_sizes=(n_feature, n_feature, 50, 25), activation='relu', max_iter = 3000, random_state = 1)
attacker_mlp_logit.fit(X_train_attacker_df, y_train_attacker_np)

y_pred_np = attacker_mlp_logit.predict(X_train_attacker_df)
accuracy  = round(np.mean(y_pred_np == y_train_attacker_np), 2)
print(f'MIA train accuracy: {accuracy}')

model_file = '../dataset/customer_attacker_mlp_logit.pkl'
joblib.dump(attacker_mlp_logit, model_file)


#=============== Attacker Testing ==================
df_member = df_train.tail(int(len(df_train)/2))
df_non_member = df_test_list[1]
df_mem_nonmem = pd.concat([df_member, df_non_member], ignore_index=True)

col_rest = df_mem_nonmem.columns[1:]
X_mem_nonmem = torch.tensor(df_mem_nonmem[col_rest].values, dtype=torch.float32)

mem_y = np.array([int(1) for i in range(len(df_member))])
nonmem_y = np.array([int(0) for i in range(len(df_non_member))])
y_test_attacker_np = np.concatenate((mem_y, nonmem_y))

logit = target_model(X_mem_nonmem) 
df_logit = pd.DataFrame(logit.detach().numpy())
scaler = StandardScaler() #94 70
# scaler = RobustScaler() #94 70
X_test_attacker_df  = pd.DataFrame(scaler.fit_transform(df_logit))

#attacker_mlp_logit is a scikit learn model that generates 0/1 labels
y_pred_np = attacker_mlp_logit.predict(X_test_attacker_df) 
accuracy  = round(np.mean(y_pred_np == y_test_attacker_np), 2)
print(f'MIA test accuracy: {accuracy}')







'''
I_nn MIA
'''

df_test = df_test.head(int(len(df_test)/5000) * (5000))
#split df_test into int(len(df_test)/5000) = 37 data frames, each having 5000 records;
df_test_list = np.split(df_test, len(df_test)/5000)
df_member = df_train.head(int(len(df_train)/2))
df_non_member = df_test_list[0]
df_mem_nonmem = pd.concat([df_member, df_non_member], ignore_index=True)

col_rest = df_mem_nonmem.columns[1:]
X_mem_nonmem = torch.tensor(df_mem_nonmem[col_rest].values, dtype=torch.float32)

mem_y = np.array([int(1) for i in range(len(df_member))])
nonmem_y = np.array([int(0) for i in range(len(df_non_member))])
y_train_attacker_np = np.concatenate((mem_y, nonmem_y))

logit = target_model(X_mem_nonmem) #logit is tensor here
X_mem_nonmem_prob = torch.sigmoid(logit)
X_train_attacker_df = pd.DataFrame(X_mem_nonmem_prob.detach().numpy())

#=============== Attacker Training ==================
n_feature = len(X_train_attacker_df.columns) 
attacker_mlp_prob = MLPClassifier(hidden_layer_sizes=(n_feature, n_feature, 50, 25), activation='relu', max_iter = 3000, random_state = 1)
attacker_mlp_prob.fit(X_train_attacker_df, y_train_attacker_np)

y_pred_np = attacker_mlp_prob.predict(X_train_attacker_df)
accuracy  = round(np.mean(y_pred_np == y_train_attacker_np), 2)
print(f'MIA train accuracy: {accuracy}')

model_file = '../dataset/customer_attacker_mlp_prob.pkl'
joblib.dump(attacker_mlp_prob, model_file)


#=============== Attacker Testing ==================
df_member = df_train.tail(int(len(df_train)/2))
df_non_member = df_test_list[1]
df_mem_nonmem = pd.concat([df_member, df_non_member], ignore_index=True)

col_rest = df_mem_nonmem.columns[1:]
X_mem_nonmem = torch.tensor(df_mem_nonmem[col_rest].values, dtype=torch.float32)

mem_y = np.array([int(1) for i in range(len(df_member))])
nonmem_y = np.array([int(0) for i in range(len(df_non_member))])
y_test_attacker_np = np.concatenate((mem_y, nonmem_y))

logit = target_model(X_mem_nonmem) 
X_mem_nonmem_prob = torch.sigmoid(logit)
X_test_attacker_df = pd.DataFrame(X_mem_nonmem_prob.detach().numpy())

#attacker_mlp_logit is a scikit learn model that generates 0/1 labels
y_pred_np = attacker_mlp_prob.predict(X_test_attacker_df) 
accuracy  = round(np.mean(y_pred_np == y_test_attacker_np), 2)
print(f' MIA test accuracy: {accuracy}')




'''
I_bl MIA 
'''

def myCustomLoss(my_outputs, my_labels):
    #specifying the batch size
    my_batch_size = my_outputs.size()[0] 
    #calculating the log of softmax values           
    my_outputs = F.log_softmax(my_outputs, dim=1)  
    #selecting the values that correspond to labels
    my_outputs = my_outputs[range(my_batch_size), my_labels] 
    #returning the results
    return my_outputs


df_test = df_test.head(int(len(df_test)/5000) * (5000))
#split df_test into int(len(df_test)/5000) = 37 data frames, each having 5000 records;
df_test_list = np.split(df_test, len(df_test)/5000)
df_member = df_train.head(int(len(df_train)/2))
df_non_member = df_test_list[0]
df_mem_nonmem = pd.concat([df_member, df_non_member], ignore_index=True)

col_0 = df_mem_nonmem.columns[0]
col_rest = df_mem_nonmem.columns[1:]
X_mem_nonmem = torch.tensor(df_mem_nonmem[col_rest].values, dtype=torch.float32)
y_mem_nonmem = torch.tensor(df_mem_nonmem[col_0].values)

logit = target_model(X_mem_nonmem) 
X_mem_nonmem_prob = torch.sigmoid(logit)
df_tmp = pd.DataFrame(X_mem_nonmem_prob.detach().numpy())

loss = myCustomLoss(logit, y_mem_nonmem)
df_loss = pd.DataFrame(loss.detach().numpy())

# scaler = StandardScaler() #94 70
scaler = RobustScaler() #94 70
df_loss  = pd.DataFrame(scaler.fit_transform(df_loss))

X_train_attacker_df = pd.concat([df_tmp, df_loss], ignore_index=True, axis=1) #axis=1 merges 2 dataframes side by side

mem_y = np.array([int(1) for i in range(len(df_member))])
nonmem_y = np.array([int(0) for i in range(len(df_non_member))])
y_train_attacker_np = np.concatenate((mem_y, nonmem_y))


#=============== Attacker Training ==================
n_feature = len(X_train_attacker_df.columns) 
attacker_mlp_prob_loss = MLPClassifier(hidden_layer_sizes=(n_feature, n_feature, 50, 25), activation='relu', max_iter = 3000, random_state = 1)
attacker_mlp_prob_loss.fit(X_train_attacker_df, y_train_attacker_np)

y_pred_np = attacker_mlp_prob_loss.predict(X_train_attacker_df)
accuracy  = round(np.mean(y_pred_np == y_train_attacker_np), 2)
print(f'MIA train accuracy: {accuracy}')

model_file = '../dataset/customer_attacker_mlp_prob_loss.pkl'
joblib.dump(attacker_mlp_prob_loss, model_file)


#=============== Attacker Testing ==================
df_member = df_train.tail(int(len(df_train)/2))
df_non_member = df_test_list[1]
df_mem_nonmem = pd.concat([df_member, df_non_member], ignore_index=True)

col_0 = df_mem_nonmem.columns[0]
col_rest = df_mem_nonmem.columns[1:]
X_mem_nonmem = torch.tensor(df_mem_nonmem[col_rest].values, dtype=torch.float32)
y_mem_nonmem = torch.tensor(df_mem_nonmem[col_0].values)

logit = target_model(X_mem_nonmem) 
X_mem_nonmem_prob = torch.sigmoid(logit)
df_tmp = pd.DataFrame(X_mem_nonmem_prob.detach().numpy())

loss = myCustomLoss(logit, y_mem_nonmem)
df_loss = pd.DataFrame(loss.detach().numpy())

# scaler = StandardScaler() #94 70
scaler = RobustScaler() #94 70
df_loss  = pd.DataFrame(scaler.fit_transform(df_loss))

X_test_attacker_df = pd.concat([df_tmp, df_loss], ignore_index=True, axis=1) #axis=1 merges 2 dataframes side by side

mem_y = np.array([int(1) for i in range(len(df_member))])
nonmem_y = np.array([int(0) for i in range(len(df_non_member))])
y_test_attacker_np = np.concatenate((mem_y, nonmem_y))

#attacker_mlp_logit is a scikit learn model that generates 0/1 labels
y_pred_np = attacker_mlp_prob_loss.predict(X_test_attacker_df) 
accuracy  = round(np.mean(y_pred_np == y_test_attacker_np), 2)
print(f'MIA test accuracy: {accuracy}')
